In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
import pandas as pd
df = pd.read_csv(r"C:\Users\Sundharesan.sk\Desktop\CVM\data\resume.csv")
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [4]:
len(df) - sum(df.duplicated())
df.drop_duplicates(inplace=True)

In [5]:
from transformers import BertModel, BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

model.eval()

c:\Users\Sundharesan.sk\Desktop\CVM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Sundharesan.sk\Desktop\CVM\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [6]:
def generate_bert_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embedding

In [7]:
import spacy
import re
nlp = spacy.load('en_core_web_sm')

def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text) 
    doc = nlp(str(text))
    preprocessed_text = []
    for token in doc:
        if token.is_punct or token.like_num or token.is_space:
            continue
        preprocessed_text.append(token.lemma_.lower().strip())
    return ' '.join(preprocessed_text)

In [8]:
import pandas as pd

df['preprocessed_resume'] = df['Resume'].apply(preprocess)

df['resume_embedding'] = df['preprocessed_resume'].apply(lambda x: generate_bert_embedding(x, tokenizer, model))


In [9]:
import faiss
import numpy as np

resume_embeddings = np.vstack(df['resume_embedding'].values)

embedding_dim = resume_embeddings.shape[1]

index = faiss.IndexFlatL2(embedding_dim)  
index.add(resume_embeddings) 


In [10]:
job_description = "python developer with knowledge on web applications and problem solving"

job_description_text = preprocess(job_description)
job_description_embedding = generate_bert_embedding(job_description_text, tokenizer, model)

top_n = 10  
distances, indices = index.search(np.array([job_description_embedding]), top_n)

top_resumes = df.iloc[indices[0]].reset_index(drop=True)


In [11]:
top_resumes

,Category,Resume,preprocessed_resume,resume_embedding
0,Java Developer,Education Details \r\nAugust 2010 to May 2017 ...,education details august to may be electronics...,"[-0.42322868, -0.12271026, 0.18326013, 0.00950..."
1,Python Developer,â¢ Operating Systems: Windows â¢ Others: MS ...,â operating systems windows â others ms excel ...,"[-0.56344444, -0.04752422, 0.14277492, 0.00036..."
2,Java Developer,Education Details \r\n BE IT pjlce\r\nJava D...,education details be it pjlce java developer j...,"[-0.2983434, -0.24799538, -0.039395902, -0.095..."
3,Python Developer,TECHNICAL PROFICIENCIES Platform: Ubuntu/Fedor...,technical proficiency platform ubuntufedoracen...,"[-0.40825474, 0.024256326, 0.3650754, 0.039877..."
4,Java Developer,SKILLS: - 1) Team leading 2) Self-motivated 3)...,skills team lead selfmotivated hard work â str...,"[-0.18689072, 0.11362449, 0.036214046, -0.0002..."
5,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education details mca ymcaust faridabad haryan...,"[-0.47299987, -0.02351636, 0.2651584, -0.04958..."
6,DotNet Developer,Education Details \r\nJanuary 2014 Education ...,education details january education details pu...,"[-0.122131236, 0.12175534, 0.34666103, -0.0230..."
7,SAP Developer,Education Details \r\nJanuary 2016 Bachelor Of...,education details january bachelor of engineer...,"[-0.6363699, 0.06071102, 0.24468774, 0.2525418..."
8,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education details may to may be uitrgpv data s...,"[-0.58747035, 0.2571115, -0.08278744, -0.11146..."
9,HR,Education Details \r\n BBA lovely profession...,education details bba lovely professional univ...,"[-0.44247127, -0.06615432, 0.018497003, -0.259..."


In [12]:
df.head()

,Category,Resume,preprocessed_resume,resume_embedding
0,Data Science,Skills * Programming Languages: Python (pandas...,skill programming languages python panda numpy...,"[-0.71765864, 0.16068065, -0.090576366, 0.1191..."
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education details may to may be uitrgpv data s...,"[-0.58747035, 0.2571115, -0.08278744, -0.11146..."
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",area of interest deep learning control system ...,"[-0.20936157, 0.17373037, 0.3383843, 0.0527605..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill â r â python â sap hana â tableau â sap ...,"[-0.7277306, 0.2818113, 0.08988865, 0.20270589..."
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education details mca ymcaust faridabad haryan...,"[-0.47299987, -0.02351636, 0.2651584, -0.04958..."


In [2]:
import os
import pandas as pd

folder_path = r'C:\Users\Sundharesan.sk\Desktop\CVM\tmptxt\CVs'
file_contents = []

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            file_contents.append({'filename': filename, 'content': content})

df = pd.DataFrame(file_contents)
df

,filename,content
0,Rahul_Resume_.txt,Rahul Bhimakari\n+91-9483849803 |GMail |Linked...
1,Varshini_Sept 2024.txt,VARSHINI JEYACHANDRAN\nAspiring Software Engin...
